In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
from tensorflow.keras.models import load_model
from multiprocessing import cpu_count
from warnings import catch_warnings
from warnings import filterwarnings
from pandas import read_csv
import matplotlib.pyplot as plt
from pandas import DataFrame
import time
#from datetime import datetime
import datetime
import itertools
import mysql.connector
from mysql.connector import connection
from mysql.connector import errorcode
from statistics import mean

In [12]:
station = ['t51']
previous_id = np.zeros(len(station))

In [13]:
def loadData(t):
    myconn = mysql.connector.connect(host = "82.180.142.51",
                                database = "u978805288_land",
                                user = "u978805288_root",
                                password= "ACSL@b123")
    (myconn)

    sql_select_query = "select sensor_id, value from data_log, info_time where data_log.info_id = info_time.info_id AND data_log.triplet_id = '"+t+"' ORDER BY data_log.info_id desc limit 60"
    date_select_query = "select Date_Time from info_time where triplet_id='"+t+"' order by info_id desc limit 1"

    cursor = myconn.cursor()
    cursor.execute(sql_select_query)
    data = cursor.fetchall()

    cursor.execute(date_select_query)
    date = cursor.fetchall()

    myconn.close()
    return data,date[0][0]


In [14]:
triplet_id = "t51"
data, date_value = loadData(triplet_id)
print("Data:")
# print(data)
for row in data:
    print(row)

print("Date:", date_value)

Data:
('s7', '563')
('s5', '-0.77,-0.01,-0.45,7.74,4.21,-5.55,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.10')
('s1', '1.40,24')
('s7', '563')
('s5', '-0.77,-0.00,-0.45,9.33,4.15,-5.85,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.14')
('s1', '1.70,34')
('s7', '562')
('s5', '-0.78,-0.01,-0.44,7.87,5.18,-4.63,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.31')
('s1', '2.00,29')
('s7', '562')
('s5', '-0.77,0.00,-0.44,8.17,6.04,-5.91,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.44')
('s1', '1.80,30')
('s7', '562')
('s5', '-0.77,-0.00,-0.44,6.40,5.43,-4.21,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.45')
('s1', '2.20,34')
('s7', '561')
('s5', '-0.77,-0.00,-0.44,7.26,5.00,-4.88,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.39')
('s1', '1.70,26')
('s7', '561')
('s5', '-0.77,-0.00,-0.44,6.83,4.88,-5.67,0.00,0.00,0.00,0')
('s4', '0.00')
('s3', '0')
('s2', '1054.48')
('s1', '1.30,29')
('s7', '561')
('s5', '-0.77,0

In [15]:
def dataPreprocess(data):
    result = []
    data.reverse()

    combined_data = []
    for i in range(0, len(data), 7):
        row = data[i:i+7]
        sensor_data = []

        for s, d in row:
            sid = s
            if sid == 's6':
                continue
            elif sid == 's5':
                x = d.split(',')[:6]
                for j in x:
                    sensor_data.append(float(j))
                x = d.split(',')[-1]
                sensor_data.append(float(x))
            elif sid.startswith('s'):
                x = d.split(',')[:]
                for j in x:
                    sensor_data.append(float(j))

        combined_data.extend(sensor_data)

    data = np.array(combined_data)
    return combined_data


preprocessed_data = dataPreprocess(data)
split_rows = [preprocessed_data[i:i+13] for i in range(0, len(preprocessed_data), 13)]

for row in split_rows:
    print(row)


[2.2, 25.0, 1054.47, 0.0, 0.0, -0.77, 0.0, -0.43, 5.67, 5.0, -4.57, 0.0, 560.0]
[1.6, 27.0, 1054.45, 0.0, 0.0, -0.77, -0.0, -0.45, 6.89, 5.12, -5.43, 0.0, 560.0]
[2.2, 27.0, 1054.42, 0.0, 0.0, -0.77, 0.0, -0.45, 6.71, 4.27, -4.76, 0.0, 561.0]
[1.3, 29.0, 1054.48, 0.0, 0.0, -0.77, -0.0, -0.44, 6.83, 4.88, -5.67, 0.0, 561.0]
[1.7, 26.0, 1054.39, 0.0, 0.0, -0.77, -0.0, -0.44, 7.26, 5.0, -4.88, 0.0, 561.0]
[2.2, 34.0, 1054.45, 0.0, 0.0, -0.77, -0.0, -0.44, 6.4, 5.43, -4.21, 0.0, 562.0]
[1.8, 30.0, 1054.44, 0.0, 0.0, -0.77, 0.0, -0.44, 8.17, 6.04, -5.91, 0.0, 562.0]
[2.0, 29.0, 1054.31, 0.0, 0.0, -0.78, -0.01, -0.44, 7.87, 5.18, -4.63, 0.0, 562.0]
[1.7, 34.0, 1054.14, 0.0, 0.0, -0.77, -0.0, -0.45, 9.33, 4.15, -5.85, 0.0, 563.0]
[1.4, 24.0, 1054.1, 0.0, 0.0, -0.77, -0.01, -0.45, 7.74, 4.21, -5.55, 0.0, 563.0]


In [16]:
import tensorflow as tf
data_list = []
count = 0
for i in range(10):
    d = preprocessed_data[count:count+13]
    data_list.append(d)
    count += 13

data_array = np.array(data_list)
data_array = data_array.reshape((-1, 10, 13))
data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
print(data_array)

[[[ 2.20000e+00  2.50000e+01  1.05447e+03  0.00000e+00  0.00000e+00
   -7.70000e-01  0.00000e+00 -4.30000e-01  5.67000e+00  5.00000e+00
   -4.57000e+00  0.00000e+00  5.60000e+02]
  [ 1.60000e+00  2.70000e+01  1.05445e+03  0.00000e+00  0.00000e+00
   -7.70000e-01 -0.00000e+00 -4.50000e-01  6.89000e+00  5.12000e+00
   -5.43000e+00  0.00000e+00  5.60000e+02]
  [ 2.20000e+00  2.70000e+01  1.05442e+03  0.00000e+00  0.00000e+00
   -7.70000e-01  0.00000e+00 -4.50000e-01  6.71000e+00  4.27000e+00
   -4.76000e+00  0.00000e+00  5.61000e+02]
  [ 1.30000e+00  2.90000e+01  1.05448e+03  0.00000e+00  0.00000e+00
   -7.70000e-01 -0.00000e+00 -4.40000e-01  6.83000e+00  4.88000e+00
   -5.67000e+00  0.00000e+00  5.61000e+02]
  [ 1.70000e+00  2.60000e+01  1.05439e+03  0.00000e+00  0.00000e+00
   -7.70000e-01 -0.00000e+00 -4.40000e-01  7.26000e+00  5.00000e+00
   -4.88000e+00  0.00000e+00  5.61000e+02]
  [ 2.20000e+00  3.40000e+01  1.05445e+03  0.00000e+00  0.00000e+00
   -7.70000e-01 -0.00000e+00 -4.40000

In [17]:
STOREDIR = ('finalmodel')
model1 = tf.saved_model.load(STOREDIR)
predict_fn = model1.signatures["serving_default"]

# Load the second model
STOR = "/content/drive/MyDrive/Count_data-LMS/urni dhank/"
model2 = tf.keras.models.load_model('website1.h5')

# model1 = tf.saved_model.load('saved_model.pb')
# predict_fn = model1.signatures["serving_default"]
# print("path: ",)
# model2 = tf.keras.models.load_model('website1.h5')

def runModel(data_tensor):
    previous = 1
    try:
        # Run the prediction on the first model
        pred = predict_fn(data_tensor)
        pred = np.argmax(pred)
        pred = np.round(pred).astype(int)

        # Run the prediction on the second model
        dt = data_tensor[:,:,0:5]
        pred1 = model2.predict(dt)

        combined_list = []
        combined_list.extend([round(x,2) for x in pred1.tolist()[0]])
        combined_list.append(pred)

        return (combined_list)
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [18]:
def insertDatabase(prediction, triplet_id):
    connection = mysql.connector.connect(host = "82.180.142.51",
                                database = "u978805288_land",
                                user =   "u978805288_root",
                                password= "ACSL@b123")
    try:
        (connection)
        current_time = date + datetime.timedelta()
        print('current_time:',current_time)

        val = "(NULL,'" +(triplet_id)+"',"+ str(prediction[0])+","+ str(prediction[1])+","+str(prediction[2])+","+str(prediction[3])+","+str(prediction[4])+","+str(prediction[5])+ ",'"+str(current_time)+ "','no')"

        add_prediction = "INSERT INTO Prediction_Real_Data  VALUES "+val
        print(add_prediction)

        cursor = connection.cursor()
        cursor.execute(add_prediction)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into prediction table")
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into prediction table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()

In [19]:
while True:
  count = 0
  for triplet_id in station:
    data,date = loadData(triplet_id)
    print(date)
    date = date+ datetime.timedelta(minutes=10)
    data = dataPreprocess(data)
    data_list = []
    count = 0
    for i in range(10):
        d = preprocessed_data[count:count+13]
        data_list.append(d)
        count += 13

    data_array = np.array(data_list)
    data_array = data_array.reshape((-1, 10, 13))
    data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
    prediction = runModel(data_tensor)
    print(prediction)
    insertDatabase(prediction,triplet_id)
    print('Enjoy')
  count+=1

2024-03-14 23:57:13
1/1 [==============================] - 1s 982ms/step
[23.86, 67.89, 984.09, 0.0, 453.39, 0]
current_time: 2024-03-15 00:07:13
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t51',23.86,67.89,984.09,0.0,453.39,0,'2024-03-15 00:07:13','no')
1 Record inserted successfully into prediction table
Enjoy
2024-03-14 23:57:13
1/1 [==============================] - 0s 50ms/step
[23.86, 67.89, 984.09, 0.0, 453.39, 0]
current_time: 2024-03-15 00:07:13
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t51',23.86,67.89,984.09,0.0,453.39,0,'2024-03-15 00:07:13','no')
1 Record inserted successfully into prediction table
Enjoy
2024-03-14 23:57:13
1/1 [==============================] - 0s 41ms/step
[23.86, 67.89, 984.09, 0.0, 453.39, 0]
current_time: 2024-03-15 00:07:13
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t51',23.86,67.89,984.09,0.0,453.39,0,'2024-03-15 00:07:13','no')
1 Record inserted successfully into prediction table
Enjoy
2024-03-14 23:57:13
1/1 [=======================

KeyboardInterrupt: 